Jonathan Kelley

SRIP 2017 Bennett

Modeling electron trajectory in FEM-generated B and E fields


This version of electron trajectory uses the Runge Kutta 4th order method to approximate
electron trajectory in magnetic fields as defined by coil loops. Ideally, the magnetic 
field can be any value as long as it can be estimated by a position in space.

The magnetic fields presented here are time-invariant, assuming steady-state operation
of the device. Currently (7.19.17) the code does not support time-dependent coil current,
but would need to if electron capture time begins to approach coil time constants.

Technically this code supports the CDecimal package, but is missing certain functions
that prohibit enabling it for the Runge Kutta method (Euler's still works). Testing
shows the the cdecimal is not needed as the RK4 method generate error above e-15 
signifigance and the cdecimal package is significantly more computationally intensive.

Program complexity is O(n) and processes 10,000 time steps every 3 seconds.

Coil formulae (r, z) pulled from:
https://tiggerntatie.github.io/emagnet/index.htm

In [1]:
# Imports
from __future__ import division
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from math import sqrt,sin,cos,atan2, sqrt, pi
from cdecimal import Decimal as D
from scipy.special import ellipk, ellipe
import pyqtgraph as pg
import pyqtgraph.opengl as gl

In [2]:
# PyQt Initialization
pg.mkQApp()
view = gl.GLViewWidget()
view.show()
view.opts['distance'] = 1

In [3]:
# Variables and Dubugger
duration_steps = 1.5e5#1e6
b_comp = np.array([0.0,0.0,.07712710231773079])

In [4]:
# Constants/Refrences
generic_xyz_coords = np.array([0,0,0])
mu_0 = D('1.25663706e-6')
ellipe_table = ellipe(np.arange(0,1, 1.0/10000000.0))
ellipk_table = ellipk(np.arange(0,1,1.0/10000000.0))
len_el_tab = float(len(ellipe_table))

In [5]:
# Currently num_turns does nothing                                  
class coil:
    def __init__(self, radius = D('0.05'), current=D('100'), position=np.array([D('0'),D('0'),D('0')]), plane_axis = 0, num_turns = 1, usedecimal = False):
        if usedecimal:
            self.radius = radius
            self.current = current
            self.position = position
            self.num_turns = num_turns
            self.B_0 = current*mu_0 / (D('2') * radius)
            self.D_one = D('1')
            self.D_four = D('4')
            self.D_pi = D('3.141592653589793238462643')

        else:
            self.radius = float(radius)
            self.current = float(current)
            self.position = np.float64(position)
            self.num_turns = int(num_turns)
            self.B_0 = self.current * float(mu_0) / (2*self.radius)
                    
        # Premath makes future calculations a bit quicker, reduces function overhead       
        self.current_2 = self.current*self.current
        self.ellipk = ellipk
        self.ellipe = ellipe
        self.atan2 = atan2
        self.sqrt = sqrt
        self.pi = pi
        self.r = []
        self.B_r_l = []
        self.k_vals = []
        
    # Assume coil is defined in XY plane.
    # Easy to change by reassigning x y z based on an axis input TODO                                                
    def get_components_from_pos(self, x,y,z, usedecimal = False):
        z = z - self.position[2] # Shift only works for vertically stacked coils
        if usedecimal:
            r = (x*x + y*y).sqrt()
            a = r/self.radius
            B = z/self.radius
            gamma = z/r
            Q = ((self.D_one + a) * (self.D_one + a) + B*B)
            k = (self.D_four * a /Q).sqrt()
            E_k = D(self.ellipe(float(k)))
            K_k = D(self.ellipk(float(k)))

            
            B_z = self.B_0 * (self.D_one)/(self.D_pi*Q.sqrt()) * (E_k * ((self.D_one) - a*a - B*B)/(Q-self.D_four*a) + K_k )
            B_r = self.B_0 * (gamma)/(self.D_pi*Q.sqrt()) * (E_k * ((self.D_one) + a*a + B*B)/(Q-self.D_four*a) - K_k )            
           
            # New functions can handle the decimal library! 
            # Jonathan Kelley 7.12.17
            B_x = x/r*B_r 
            B_y = y/r*B_r
                        
        else:            
            r = self.sqrt(x*x + y*y)
            a = r/self.radius
            B = z/self.radius
            gamma = z/r 
            Q = ((1.0+a) * (1.0+a) + B*B)
#            k = self.sqrt(4*a/Q)
            k = (4.0*a/Q)
                    
         #   self.k_vals.append(k)          
         #   print(k,x,y,a,Q,z,r, k*len_el_tab)
            if False:
                E_k = ellipe_table[int(k*len_el_tab)]
                K_k = ellipk_table[int(k*len_el_tab)]
            else:
                E_k = self.ellipe(k)
                K_k = self.ellipk(k)    
            
            B_z = self.B_0 * (1.0)/(self.pi*self.sqrt(Q)) * ( (E_k * (1.0 - a*a - B*B)/(Q-4.0*a)) + K_k )
            B_r = self.B_0 * (gamma)/(self.pi*self.sqrt(Q)) * (E_k * (1.0 + a*a + B*B)/(Q-4.0*a) - K_k )            
            
            B_x = x/r*B_r 
            B_y = y/r*B_r
            self.r.append(r)
            self.B_r_l.append(B_r)
            
        return np.array([B_x, B_y, B_z])

In [6]:
# Does not support E fields yet
class particle:
    def __init__(self, mass = 0, charge = 0, position = generic_xyz_coords, velocity = generic_xyz_coords, acceleration = generic_xyz_coords, positions = [], bfields=[], input_energy = 0, velocities = [], usedecimal = False, time_step = D('.000000000001')):
        self.usedecimal = usedecimal
        if usedecimal:            
            self.mass = mass
            self.velocity = velocity
            self.position = position
            self.acceleration = acceleration
            self.charge = charge
            self.positions = []
            self.input_energy = input_energy
            self.velocities = []
            self.dt = time_step   
            self.step = 0  
            self.bfields = []               
        else:
            self.mass = np.float64(mass)
            self.velocity = np.float64(velocity)
            self.position = np.float64(position)
            self.acceleration = np.float64(acceleration)
            self.charge = np.float64(charge)
            self.positions = [] 
            self.input_energy = input_energy
            self.velocities = []
            self.dt = np.float64(time_step)
            self.step = 0
            self.bfields = []
            
            
    def get_accel_from_field(self, pos, coils, dt):
        
        a = self.velocity

        # Netwon Force; Lorentz Force; Acceleration
        b = np.array([x.get_components_from_pos(pos[0],pos[1],pos[2]) for x in coils])
        b = [ sum(b[:,0]),sum(b[:,1]),sum(b[:,2])]
        
        #b = b_comp # Enable this line to override the magnetic field with a constant value, useful for tuning integrators
                
        return np.array([a[1]*b[2] - a[2]*b[1], a[2]*b[0] - a[0]*b[2], a[0]*b[1] - a[1]*b[0]]) * self.charge / self.mass # Returns acceleration

In [7]:
class canvas:
    def __init__(self): 
        self.particles = []                     
        self.fig = plt.figure(1)
        plt.ion() # Interactive plotting
        self.axes = plt.gca(projection='3d')        
        self.axes.set_xlabel('X axis')
        self.axes.set_ylabel('Y axis')
        self.axes.set_zlabel('Z axis') 
        self.axes.set_title('Electron Trajectory in Magnetic Fields')
        
    def bind_particle(self, particle):
        self.particles.append(particle)    
        self.electron = particle # remove for PIC 
#        self.electron.positions = []
    
    
    def update_frame(self, dt, coils):
        
        if(False):
            # Euler's Method (1st Order)
            # Jonathan Kelley
            # July 12, 2017
            x = self.electron.position[0]
            y = self.electron.position[1]
            z = self.electron.position[2]
                                                                        
            b_comp_1 = coils[0].get_components_from_pos(x,y,z,self.electron.usedecimal)
            b_comp_2 = coils[1].get_components_from_pos(x,y,z,self.electron.usedecimal)
            b_comp = b_comp_1 + b_comp_2
            
            
            a = self.electron.velocity
            b = b_comp
            accel = np.array([a[1]*b[2] - a[2]*b[1], a[2]*b[0] - a[0]*b[2], a[0]*b[1] - a[1]*b[0]]) * self.electron.charge / self.electron.mass
                                                                            
            velocity = accel * dt      
            self.electron.velocity = self.electron.velocity + velocity
            
            new_pos = (self.electron.velocity*dt) + self.electron.position
            self.electron.position = new_pos
            self.electron.positions.append(new_pos)
            self.electron.velocities.append(self.electron.velocity)
            self.electron.bfields.append(b_comp)        
        
        else:
            # Runge Kutta 4th Order Method 
            # Jonathan Kelley
            # July 18, 2017
            
            k1 = dt * self.electron.get_accel_from_field(self.electron.position, coils, self.electron.dt) #velocity
            l1 = dt * self.electron.velocity
            
            k2 = dt * self.electron.get_accel_from_field((self.electron.position + l1/2),coils, self.electron.dt ) #Velocity at that point in time
            l2 = dt * (self.electron.velocity + k1/2)
          
            k3 = dt * self.electron.get_accel_from_field((self.electron.position + l2/2),coils, self.electron.dt ) #Velocity at that point in time
            l3 = dt * (self.electron.velocity + k2/2)
            
            k4 = dt * self.electron.get_accel_from_field((self.electron.position + l3),coils, self.electron.dt ) #Velocity at that point in time
            l4 = dt * (self.electron.velocity + k3)
            
            self.electron.velocity = self.electron.velocity + (k1+2*k2+2*k3+k4)/6.0
            self.electron.position = self.electron.position + (l1+2*l2+2*l3+l4)/6.0
            graph.electron.velocities.append(self.electron.velocity)
            graph.electron.positions.append(self.electron.position)

In [8]:
electron = particle(mass = D('9.10938356e-31'), 
                    charge = D('-1.60217662e-19'),
                    position = np.array([ D('0.001'), D('.0001'),D('0.0')]),
                    velocity = np.array([ D('-1e2'),D('0'),  D('1e6')  ]),
                    acceleration =  np.array([ D('.0'), D('.0'),D('0.0')]),
                    input_energy = 10, #KeV
                    positions = [],
                    velocities = [],
                    bfields = [],
                    usedecimal = False,
                    time_step = D('.0000000000002')           #D('.000000000000045')           
                    )

first_coil = coil(radius = D('0.035'),
                  current =D('10000'), # Currently num turns does nothing!! 
                  position=np.array([D('0'),D('0'),D('0')]), 
                  plane_axis = 0, 
                  num_turns = 1, 
                  usedecimal = False
                  )
                 
second_coil = coil(radius = D('0.035'),
                  current =D('-10000'), # Currently num turns does nothing!! 
                  position=np.array([D('0'),D('0'),D('0.05')]), 
                  plane_axis = 0, 
                  num_turns = 1, 
                  usedecimal = False
                  )